#  janomeとmarkovifyを用いた場合

今回は、形態素解析を行うためのライブラリ「janome」と<br>マルコフ連鎖を作成するためのライブラリ「markovify」を使用します。<br>

インストールされていない場合は、anacondaで使用する仮想環境を立ち上げ、<br>
下記のコードを入力し、インストールをお願いします。

(2021年4月5日現在)<br>
conda install -c conda-forge janome -y<br>
conda install -c conda-forge markovify -y

# 必要なライブラリの読み込み

必要なライブラリを読み込みます。

In [5]:
from janome.tokenizer import Tokenizer
import markovify

# データの読み込み

データを読み込みます。CSVではなく、テキストファイル形式のデータを用います。<br>

open関数を用いて、まずはファイルを開きます。<br>
open関数では、モードを指定してあげると、データを読み込むために開きますよ、<br>
データを書き込むために開きますよと指定してあげることができます。<br>

今回は、データを読み込むために開くので「r」と指定してあげます。<br>
そして文字エンコーディングの指定は「utf-8」と入力します。<br>

readメソッドでファイルに含まれるテキストを全て読み込みます。

In [6]:
file = 'df_honkirin.txt' 
f = open(file, 'r', encoding="utf-8")
text = f.read()

In [7]:
text

'喉越しが良いから\n自分に合っている\n一番美味しいから\nとてもおいしいから。\n美味しいから\n安くて美味しい\n本格的で、味を含めて好きです\n他にはない味わいだから\n美味しいから\n生ビールに近い味わいだから\nキリンビールが好きだから\n特に無し\nより味がビールに近いと思うから。\n味が美味しくビールを飲んでいるのと同じ味わいが楽しめるため\n飽きのない味だから。\n濃さがある\n飲みやすいから\nビールに負けじと味が美味しい。第三のビールなので値段が安い\nおいしいから\n美味しいから\n味が濃厚で美味しいから\n本物\nとにかく美味しい。味が本格的。\n旨い\n美味しい\n美味しいと思いますよ\n一度飲んでみて、おいしかったから。飲みやすい。高級感がある。\nおいしいから\n値段の割に味がおいしいから。\n好みだから\n味\n味がいいから\n手頃な価格で飲みやすく飽きにくい\n好み\n本格的な味わいでビールのようであるため\nキャンペーンがやっていて二度おいしい\n味が整っている\n美味しいから\nおいしい\n飲み飽きない味。\n値段が手ごろだから\n新ジャンルの中では、アルコール度数が高く、程よく酔えるので。\n苦味とコクがしっかりあって、グッとくるのどごしの良さを感じる。ほどよいキレで後味の余韻もいい。\n特にありません\nいつでもどこでも買える安心感がある\nお試しで飲んだら良かったから\nビールと遜色のない味なので\n安い割には美味しいから\nおいしいからと、その上安いからですね。\nスーパードライをいつも買っているが、代替品を探していて、本麒麟を見つけ、少し品質の良いビールと感じたので買いました。\nおいしい\n飲みやすくて香りがよいから。\nタモリが出演しているコマーシャルを見て買ったら美味しかったから\n美味しい\nビールより安く買えるのにビールに近いくらい美味しいから\n美味いから\nのどごしが良くて、飲んだ後の爽快感と爽やかな感じが良く、ビールとして全体的にバランスが取れている。\n美味しいから。\n安い上、飲みごたえがあるから\nコクがある。食事に合う。程よい苦味がある。\nとにかくコスパが良いからです。\n新ジャンルの中では麦の苦みとコクがしっかり感じられて、ビールに近い味わいで本格感がある。CMや広告でも話題で売れているの

# 前処理

textをmarkovifyで読み取れるように前処理を行う関数です。

In [8]:
def text_split(text):
    # textをmarkovifyで読み取れるように前処理を行う関数です。
    
    # 引数：text
    # 引数の型：str
    
    # 戻り値：splitted_text_str
    # 引数の型：str
    
    # 今回は複数の文字列を一回で置換できるようにします。
    # maketransで置換する文字列の置き換え表を作成して、
    # str_tableという変数に入れる
    
    str_table = str.maketrans({
        # markovifyで読み取れるよう該当する文字の置換。
        # https://github.com/jsvine/markovify/issues/84
        # アンケートデータ内に「。」がついているものとついていないものがあります。
        # 表記を統一するため一旦、「。」を削除し、
        # 「'\n'」を「。」で置換する。文末が「。」で終わるように統一します。
        
        # 例：句点「。」がついているものと、ついていないものがあります。
        # 喉 越しが良いから。\n　自分に合っている\n
        # 。を削除　↓
        # 喉越しが良いから\n　自分に合っている\n
        #　\nを。で置換
        # 喉越しが良いから。　自分に合っている。
        
        #文字列の置き換え表　
        #変換前 : 変換後
        '。': '',   
        '\n': '。',
        '\r': '',
        '(': '（',
        ')': '）',
        '[': '［',
        ']': '］',
        '"':'”',
        "'":"’",
    })
    # 文字列をstr_tableの情報を用いて置換します。
    text = text.translate(str_table)
    
    # textを単語分割（文章を形態素で分ける）
    # wakati=Trueで分かち書き（単語分割）モードにできるのでこれを利用して、
    # 戻り値、文字列 (str) のリストを返します。
    # 例：['分かち書き', 'モード', 'が', 'つき', 'まし', 'た', '！']
    
    t = Tokenizer()
    tokens = t.tokenize(text, wakati=True)
    
    # splitted_text_listを用意します。
    splitted_text_list = []
    # 分かち書きされているtokensを一つずつ処理していき
    # 「。」や感嘆符でなければ、文字の後にスペース、
    # 「。」や感嘆符であれば、「'\n'」に置換
    # splitted_text_listに連結。
    # リストの要素を連結し、一つの文字列にして返します。
    for i in tokens:
        if i != '。' and i != '！' and i != '？':
            i += ' '
        elif i == '。' or i == '！' or i == '？':
            i = '\n'
        splitted_text_list.append(i)
        splitted_text_str = "".join(splitted_text_list)
            
    return splitted_text_str

# 前処理実行後の戻り値の確認

下記を実行すると、各文章が単語に分割され文末に改行が入っている状態の一つの文字列が返ってきます。

In [9]:
text_split(text)

'喉 越し が 良い から \n自分 に 合っ て いる \n一番 美味しい から \nとても おいしい から \n美味しい から \n安く て 美味しい \n本格 的 で 、 味 を 含め て 好き です \n他 に は ない 味わい だ から \n美味しい から \n生ビール に 近い 味わい だ から \nキリンビール が 好き だ から \n特に 無し \nより 味 が ビール に 近い と 思う から \n味 が 美味しく ビール を 飲ん で いる の と 同じ 味わい が 楽しめる ため \n飽き の ない 味 だ から \n濃 さ が ある \n飲み やすい から \nビール に 負け じ と 味 が 美味しい 第 三 の ビール な ので 値段 が 安い \nおいしい から \n美味しい から \n味 が 濃厚 で 美味しい から \n本物 \nとにかく 美味しい 味 が 本格 的 \n旨い \n美味しい \n美味しい と 思い ます よ \n一度 飲ん で み て 、 おいしかっ た から 飲み やすい 高級 感 が ある \nおいしい から \n値段 の 割 に 味 が おいしい から \n好み だ から \n味 \n味 が いい から \n手頃 な 価格 で 飲み やすく 飽き にくい \n好み \n本格 的 な 味わい で ビール の よう で ある ため \nキャンペーン が やっ て い て 二 度 おいしい \n味 が 整っ て いる \n美味しい から \nおいしい \n飲み 飽き ない 味 \n値段 が 手ごろ だ から \n新 ジャンル の 中 で は 、 アルコール 度数 が 高く 、 程よく 酔える ので \n苦味 と コク が しっかり あっ て 、 グッ と くる のどごし の 良 さ を 感じる ほど よい キレ で 後味 の 余韻 も いい \n特に あり ませ ん \nいつ でも どこ でも 買える 安心 感 が ある \nお 試し で 飲ん だら 良かっ た から \nビール と 遜色 の ない 味 な ので \n安い 割 に は 美味しい から \nおいしい から と 、 その 上 安い から です ね \nスーパードライ を いつも 買っ て いる が 、 代替 品 を 探し て い て 、 本 麒麟 を 見

# N階マルコフ連鎖で文章生成

【下記、公式ドキュメントを日本語訳にしたもの】<br>
「Markovify は、大きくて句読点の整ったテキストに最適に動作します。<br>
テキストが文の区切りに .s を使用していない場合は、各文を改行して、<br>
markovify.Text クラスの代わりに markovify.NewlineText クラスを使用してください。」
<br>
<br>
今回は各文を「\n」で改行しているので「markovify.NewlineText」を使用します。<br>
NewlineTextにtextを入れてモデルを作成します。<br>

state_sizeでは「◯階」の「◯」を指定し、何単語ずつのブロックに分けるか指定します。<br>
今回は2単語ずつのブロックに分けるため「state_size=2」と指定（※デフォルトも2）。<br>
これで「2階マルコフ連鎖のモデルを作りますよ」という意味となる。<br>

In [10]:
splitted_text_str = text_split(text)
text_model = markovify.NewlineText(splitted_text_str, state_size=2)

text_modelモデルでmake_sentenceメソッドを用いて文章を生成します。<br>

make_sentenceメソッドでは、元の文章と重ならないように、1回の呼び出しで何回試行を行うか、<br>
tries（トライズ）で試行回数を指定することができます。<br>
ここでは一旦、triesに100と入れています。（10回でも20回でもOK）。<br>

for i in range(10)で作成する文章の数を決めます。<br>
今回はとりあえず10個生成してみましょう。

printで「"---------------------------------"」と記載しています。<br>
これは生成された文章がまとまって出てきて、どこが1文かわかりづらい状態で出力されるため、<br>
生成された文章を1文ずつ区切るために使用しています。<br>

In [11]:
for i in range(10):
    print(text_model.make_sentence(tries=100))
    print("---------------------------------")

頑張っ て ビール に 近い 味わい で 上質 な 苦味 が 苦手 な 方 な 自分 に 合っ て いる
---------------------------------
味 が ビール に 近い と 感じ た から 味 が 良い こと コスパ も 良い し 美味しい
---------------------------------
キリンビール が 好き です
---------------------------------
安く 好み の 味 で コク が しっかり 感じ られ て 、 ビール の よう で ある ため
---------------------------------
味 が ビール に 近い 本格 感 が あっ て 喉 越し や 後味 等 が スッキリ し て い て 、 飲ん だら コク が しっかり あっ て 味 が 濃い から
---------------------------------
自分 の 口 に 合う ビール と 感じ た の が 、 代替 品 を 探し て い て 飲み やすかっ た から
---------------------------------
一度 飲ん で いる の と 同じ で あり 満足 し て いる
---------------------------------
本格 的 な 味わい で 本格 的
---------------------------------
飲み ごたえ が あっ て 飲み たい と 思っ た から
---------------------------------
低 価格 の わり に 本格 的 な 味わい で ビール の よう で ある 事
---------------------------------


In [12]:
text_model_3 = markovify.NewlineText(splitted_text_str, state_size=3)

In [13]:
for i in range(10):
    print(text_model_3.make_sentence(tries=100))
    print("---------------------------------")

普段 は 一番 搾り を 飲む が 、 本 麒麟 を 見つけ 、 少し 品質 の 良い ビール と 感じ た から 味 が 濃い から
---------------------------------
飲み ごたえ が あっ て 味 が 美味しい 第 三 の ビール な のに 、 ビール の よう で ある ため
---------------------------------
キリン の 力 を 入れ て いる 製品 だけ あり 、 コク が あり クセ が 無く 飲み やすい
---------------------------------
ビール より 安く 買える のに ビール に 近い 本格 的 な ので
---------------------------------
美味しい から 味 が ビール に 近い 味わい な ので
---------------------------------
コク が あっ て 美味しい
---------------------------------
喉 越し が よく 飲み 飽き ない 味 が 気に入っ て い ます ので 大好き です
---------------------------------
ビール に 近い 本格 的 な ビール と 同じ で あり 満足 し て いる
---------------------------------
喉 越し や 後味 等 が スッキリ し て い ます ので 大好き です
---------------------------------
第 三 の ビール な のに 、 ビール に 近く て おいしい から
---------------------------------


# 注意点

<b>＜注意点１＞</b><br>
モデルを実行する度に、新しい文章が生成されます。<br>
作成した文章を保存しておく必要があれば保存しておきます<br>
（手動でコピーをする、またはプログラムを実行し保存するなど）。
  
<b>＜注意点２＞</b><br>
本来、自然言語処理を行う場合は、形態素に分解（単語分割）→言葉を原形に変換→<br>
ストップワード（助詞の「は」「を」「に」や感動詞「あぁ」など）の除去などの前処理を基本的に行って<br>解析精度を向上させる方法を取ります。<br>

しかし今回の場合は、アンケートで頂いた「生の声」を自動生成した文章にも反映したかったため、<br>
上記の前処理の工程では、形態素に分解（単語分割）までに留めておき、<br>
言葉を原形に変換することや、ストップワードの除去などは行っていません。	